In [ ]:
!pip install catboost
!pip install optuna
#!pip install lightgbm --install-option=--gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from functools import partial

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model
from sklearn import ensemble
from sklearn import svm
import xgboost as xgb
import lightgbm as lgbm
from catboost import CatBoostClassifier

import gc
import optuna
from datetime import datetime

In [ ]:
def apk(actual, predicted, k=10):
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
ori = pd.read_csv('trainn.csv')
train.drop('id', inplace=True, axis=1)
test.drop('id', inplace=True, axis=1)
train = pd.concat([train,ori])

In [ ]:
enc = OrdinalEncoder()
train['prognosis'] = enc.fit_transform(train[['prognosis']])
swapped_map = enc.inverse_transform(np.array([i for i in range(11)]).reshape(-1,1))
swapped_map = [s[0] for s in swapped_map]

In [ ]:
target = train['prognosis']
train.drop('prognosis', inplace=True, axis=1)

In [ ]:
lmao = """
for i in range(test_predss.shape[0]):
    rank_list = list(rank_df.iloc[i, :].sort_values(0, ascending=False).iloc[:3].index)
    sub.loc[i, 'prognosis'] = ' '.join([swapped_map[_] for _ in rank_list])

# Save submission file
sub.to_csv('ensemble.csv', index=False)
sub
"""

## Feature Engineering


---



In [ ]:
for df in [train, test]:
    columns = [col for col in df if col != 'prognosis']
    df[columns] = df[columns].astype(int)

    df['jumlah'] = df[columns].sum(axis=1)

    similar_columns = ['loss_of_appetite', 'urination_loss', 'slow_heart_rate', 'abdominal_pain', 'light_sensitivity', 'yellow_skin', 'yellow_eyes']
    df['similar_cluster'] = df[similar_columns].sum(axis=1)

    lyme_columns = ['jaundice', 'weight_loss', 'weakness', 'back_pain', 'sudden_fever', 'myalgia', 'chills', 'orbital_pain', 'digestion_trouble']
    df['lyme_cluster'] = df[lyme_columns].sum(axis=1)

    chikungunya_columns = ['convulsion', 'finger_inflammation', 'speech_problem', 'toenail_loss', 'ulcers', 'itchiness', 'lips_irritation', 'breathing_restriction', 'toe_inflammation', 'paralysis', 'stomach_pain', 'confusion', 'irritability', 'bullseye_rash']
    df['chikungunya_cluster'] = df[chikungunya_columns].sum(axis=1)

    cluster_0 = df.columns[df.columns.str.contains('pain')]
    df['cluster_0'] = df[cluster_0].sum(axis=1)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train, target, random_state = 42)

## Hyperparameter Tuning


---




In [ ]:
def obj(trial):
    params = {"n_jobs": -1,
              "verbose": -1,
              "force_col_wise": True,
              "n_estimators": trial.suggest_int("n_estimators", 8300,8500),
              "boosting_type": 'gbdt',
              "max_bin": 251,
              "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1),
              "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1.0),
              "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1.0),
              "max_depth": trial.suggest_int("max_depth", 2, 25),
              "num_leaves": trial.suggest_int("num_leaves", 40,50),
              "colsample_bytree":trial.suggest_float("colsample_bytree",0.8, 0.9),
              "subsample":trial.suggest_float("subsample", 0.8,0.9),
              "subsample_freq": 100
    }
    model = lgbm.LGBMClassifier(**params)
    return -cross_val_score(model, train, target, cv=5, scoring="neg_root_mean_squared_error").mean()

study_lgbm = optuna.create_study(direction = 'minimize')
study_lgbm.optimize(obj, n_trials=100)

[I 2023-05-01 08:40:57,647] A new study created in memory with name: no-name-1d5386ef-0417-41e5-b5b3-d976d4313a2e
<ipython-input-10-8b4b43053772>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1.0),
<ipython-input-10-8b4b43053772>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1.0),
[I 2023-05-01 08:41:51,374] Trial 0 finished with value: 0.45208333333333334 and parameters: {'n_estimators': 8411, 'learning_rate': 0.0026017208309770936, 'reg_alpha': 3.1797707408874657e-07, 'reg_lambda': 0.04950403176228687, 'max_depth': 8, 'nu

In [ ]:
LGBMModel = lgbm.LGBMClassifier(**study_lgbm.best_params)
LGBMModel.fit(train, target)
sub_3_pred = LGBMModel.predict_proba(test)
top_3_pred = np.argsort(-sub_3_pred, axis=1)[:,:3]
top_3_pred = enc.inverse_transform(top_3_pred.reshape(-1, 1))
top_3_pred = top_3_pred.reshape(303,3)
top_3_pred[:10]
pred_sub_3 = np.array([])
for i in range(303):
  pred_sub_3 = np.append(pred_sub_3, top_3_pred[i][0] + ' ' + top_3_pred[i][1] + ' ' + top_3_pred[i][2])
sub['prognosis'] = pred_sub_3

In [ ]:
sub

,id,prognosis
0,707,Tungiasis Dengue Chikungunya
1,708,Dengue Plague Malaria
2,709,West_Nile_fever Japanese_encephalitis Lyme_dis...
3,710,Japanese_encephalitis Dengue Tungiasis
4,711,West_Nile_fever Malaria Yellow_Fever
...,...,...
298,1005,West_Nile_fever Yellow_Fever Zika
299,1006,Malaria Lyme_disease Plague
300,1007,Plague Lyme_disease West_Nile_fever
301,1008,Rift_Valley_fever West_Nile_fever Yellow_Fever


In [ ]:
sub.to_csv('lgbm.csv', columns=['id', 'prognosis'], index=False)

In [ ]:
def tuner(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 7500, 12000, step=100),
        "max_depth":trial.suggest_int("max_depth", 4, 20),
        "min_child_weight": trial.suggest_int("min_child_weight", 7, 8),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 10.),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10.),
        "gamma": trial.suggest_float("gamma", 0.7, 1.0, step=0.1),
    }

    model = xgb.XGBClassifier(
        **params,
        n_jobs=-1,
        objective="reg:squarederror",
        tree_method='gpu_hist',
        predictor = 'gpu_predictor',
        gpu_id = 0,
        enable_categorical = True
    )

    model.fit(
        x_train,
        y_train,
        early_stopping_rounds=500,
        eval_set=[(x_test,y_test)],
        verbose=0)

    pred_xg = model.predict_proba(x_test)
    top_3 = np.argsort(-pred_xg, axis=1)[:,:3]
    score = mapk(y_test.values.reshape(-1,1), top_3, k=3)

    return score

study_xgb = optuna.create_study(direction = 'maximize')
study_xgb.optimize(tuner, n_trials=25)

[I 2023-05-01 09:48:50,184] A new study created in memory with name: no-name-deac085d-0f44-42e5-83b6-9f29df5b1443
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-05-01 09:50:56,815] Trial 0 finished with value: 0.4513888888888889 and parameters: {'n_estimators': 12000, 'max_depth': 8, 'min_child_weight': 7, 'learning_rate': 0.00014657623040495189, 'subsample': 0.738303243533079, 'colsample_bytree': 0.6096316814226338, 'reg_alpha': 0.441142883167281, 'reg_lambda': 4.1250334504731985, 'gamma': 0.8999999999999999}. Best is trial 0 with value: 0.4513888888888889.
[I 2023-05-01 09:51:04,252] Trial 1 finished with value: 0.46458333333333335 and parameters: {'n_estimators': 11500, 'max_depth': 11, 'min_child_weight': 8, 'learning_rate': 0.04316023712034764, 'subsample': 0.72904

In [ ]:
final_xgb_params = study_xgb.best_params
XGBmodel = xgb.XGBClassifier(
    **final_xgb_params,
    n_jobs=-1,
    tree_method='gpu_hist'
)

XGBmodel.fit(
    train,
    target,
    early_stopping_rounds=300,
    eval_set=[(train,target)],
    verbose=0)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4887610485372076, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.8999999999999999, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.011259993839517994, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              n_estimators=8300, n_jobs=-1, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
sub_3_pred = XGBmodel.predict_proba(test)
top_3_pred = np.argsort(-sub_3_pred, axis=1)[:,:3]
top_3_pred = enc.inverse_transform(top_3_pred.reshape(-1, 1))
top_3_pred = top_3_pred.reshape(303,3)
top_3_pred[:10]
pred_sub_3 = np.array([])
for i in range(303):
  pred_sub_3 = np.append(pred_sub_3, top_3_pred[i][0] + ' ' + top_3_pred[i][1] + ' ' + top_3_pred[i][2])
sub['prognosis'] = pred_sub_3

In [ ]:
sub.to_csv('xgb.csv', columns=['id', 'prognosis'], index=False)

In [ ]:
sub

,id,prognosis
0,707,Chikungunya Dengue Zika
1,708,Dengue Chikungunya Rift_Valley_fever
2,709,West_Nile_fever Zika Japanese_encephalitis
3,710,Japanese_encephalitis Dengue Zika
4,711,West_Nile_fever Japanese_encephalitis Zika
...,...,...
298,1005,Zika Yellow_Fever Plague
299,1006,Malaria Lyme_disease Plague
300,1007,Lyme_disease Plague West_Nile_fever
301,1008,Rift_Valley_fever West_Nile_fever Dengue


In [ ]:
def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    model.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)
    top_3 = np.argsort(-y_pred, axis=1)[:,:3]
    return mapk(y_test.values.reshape(-1,1), top_3, k=3)

In [ ]:
study_cb = optuna.create_study(study_name="catboost", direction="maximize")
study_cb.optimize(objective, n_trials=25)

[I 2023-05-01 10:17:06,001] A new study created in memory with name: catboost
[I 2023-05-01 10:18:10,044] Trial 0 finished with value: 0.4506944444444444 and parameters: {'iterations': 524, 'learning_rate': 0.007611535466066916, 'depth': 8, 'l2_leaf_reg': 9.740409440004395e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.9332786349278316e-06, 'bagging_temperature': 6.5309346736786456, 'od_type': 'Iter', 'od_wait': 14}. Best is trial 0 with value: 0.4506944444444444.
[I 2023-05-01 10:18:41,087] Trial 1 finished with value: 0.45694444444444443 and parameters: {'iterations': 510, 'learning_rate': 0.0013835886147392548, 'depth': 7, 'l2_leaf_reg': 1.0212366984458243e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 7.877786525381709e-06, 'bagging_temperature': 1.1929684554256959, 'od_type': 'IncToDec', 'od_wait': 11}. Best is trial 1 with value: 0.45694444444444443.
[I 2023-05-01 10:18:50,970] Trial 2 finished with value: 0.45486111111111105 and parameters: {'iterations': 340, 'l

In [ ]:
CBModel = CatBoostClassifier(**study_cb.best_params)
CBModel.fit(train, target)
sub_3_pred = CBModel.predict_proba(test)
top_3_pred = np.argsort(-sub_3_pred, axis=1)[:,:3]
top_3_pred = enc.inverse_transform(top_3_pred.reshape(-1, 1))
top_3_pred = top_3_pred.reshape(303,3)
top_3_pred[:10]
pred_sub_3 = np.array([])
for i in range(303):
  pred_sub_3 = np.append(pred_sub_3, top_3_pred[i][0] + ' ' + top_3_pred[i][1] + ' ' + top_3_pred[i][2])
sub['prognosis'] = pred_sub_3
sub.to_csv('cb.csv', columns=['id', 'prognosis'], index=False)

0:	learn: 2.3947885	total: 7.99ms	remaining: 7.87s
1:	learn: 2.3922542	total: 15.8ms	remaining: 7.79s
2:	learn: 2.3882971	total: 23.3ms	remaining: 7.64s
3:	learn: 2.3859598	total: 30.6ms	remaining: 7.5s
4:	learn: 2.3827049	total: 38ms	remaining: 7.46s
5:	learn: 2.3795222	total: 46ms	remaining: 7.51s
6:	learn: 2.3771121	total: 53.6ms	remaining: 7.5s
7:	learn: 2.3735477	total: 62ms	remaining: 7.58s
8:	learn: 2.3709663	total: 69.6ms	remaining: 7.55s
9:	learn: 2.3669947	total: 77ms	remaining: 7.52s
10:	learn: 2.3642937	total: 85.2ms	remaining: 7.55s
11:	learn: 2.3611827	total: 92.3ms	remaining: 7.49s
12:	learn: 2.3574308	total: 99.8ms	remaining: 7.47s
13:	learn: 2.3548567	total: 107ms	remaining: 7.44s
14:	learn: 2.3521700	total: 115ms	remaining: 7.42s
15:	learn: 2.3493394	total: 122ms	remaining: 7.41s
16:	learn: 2.3467236	total: 130ms	remaining: 7.39s
17:	learn: 2.3445295	total: 137ms	remaining: 7.37s
18:	learn: 2.3411195	total: 144ms	remaining: 7.35s
19:	learn: 2.3383229	total: 152ms	rema

In [ ]:
study_lgbm.best_trials

In [ ]:
study_cb.best_trials

In [ ]:
study_xgb.best_trials

In [ ]:
def objsvc(trial):
    params = {
        'C' : trial.suggest_float("C", 1e-2, 1e2, log=True),
        'gamma' : trial.suggest_float("gamma", 1e-2, 1e-2, log=True),
        'kernel' : trial.suggest_categorical("kernel", ["linear", "poly", "rbf"])
    }
    model = svm.SVC(**params, probability=True)
    model.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)
    top_3 = np.argsort(-y_pred, axis=1)[:,:3]

    return mapk(y_test.values.reshape(-1,1), top_3, k=3)

study_svc = optuna.create_study(direction = 'maximize')
study_svc.optimize(objsvc, n_trials=500)

[I 2023-05-01 10:49:15,369] A new study created in memory with name: no-name-ffed4102-0e1d-4b12-bff0-00bc24e3eeed
[I 2023-05-01 10:49:15,728] Trial 0 finished with value: 0.4284722222222222 and parameters: {'C': 0.10259511808556476, 'gamma': 0.01, 'kernel': 'rbf'}. Best is trial 0 with value: 0.4284722222222222.
[I 2023-05-01 10:49:19,077] Trial 1 finished with value: 0.3993055555555555 and parameters: {'C': 24.63138952587405, 'gamma': 0.01, 'kernel': 'linear'}. Best is trial 0 with value: 0.4284722222222222.
[I 2023-05-01 10:49:19,540] Trial 2 finished with value: 0.39999999999999997 and parameters: {'C': 0.3070553939788756, 'gamma': 0.01, 'kernel': 'poly'}. Best is trial 0 with value: 0.4284722222222222.
[I 2023-05-01 10:49:19,853] Trial 3 finished with value: 0.4319444444444444 and parameters: {'C': 0.06325807460569073, 'gamma': 0.01, 'kernel': 'rbf'}. Best is trial 3 with value: 0.4319444444444444.
[I 2023-05-01 10:49:20,174] Trial 4 finished with value: 0.40277777777777773 and par

In [ ]:
study_svc.best_trials

In [ ]:
model_xgb = xgb.XGBClassifier(
    **study_xgb.best_params,
    n_jobs=-1,
    tree_method='gpu_hist'
)

model_xgb.fit(
    train,
    target,
    early_stopping_rounds=300,
    eval_set=[(train,target)],
    verbose=0)

pred_xgb = model_xgb.predict_proba(test)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [ ]:
model_cb = CatBoostClassifier(**study_cb.best_params)
model_cb.fit(train, target)
pred_cb = model_cb.predict_proba(test)

0:	learn: 2.3947885	total: 8.2ms	remaining: 8.08s
1:	learn: 2.3922542	total: 15.9ms	remaining: 7.84s
2:	learn: 2.3882971	total: 23.4ms	remaining: 7.68s
3:	learn: 2.3859598	total: 30.7ms	remaining: 7.53s
4:	learn: 2.3827049	total: 38.5ms	remaining: 7.56s
5:	learn: 2.3795222	total: 46.4ms	remaining: 7.58s
6:	learn: 2.3771121	total: 54ms	remaining: 7.55s
7:	learn: 2.3735477	total: 61.7ms	remaining: 7.54s
8:	learn: 2.3709663	total: 69.1ms	remaining: 7.5s
9:	learn: 2.3669947	total: 76.8ms	remaining: 7.49s
10:	learn: 2.3642937	total: 83.9ms	remaining: 7.44s
11:	learn: 2.3611827	total: 91.4ms	remaining: 7.42s
12:	learn: 2.3574308	total: 99.1ms	remaining: 7.42s
13:	learn: 2.3548567	total: 107ms	remaining: 7.42s
14:	learn: 2.3521700	total: 114ms	remaining: 7.39s
15:	learn: 2.3493394	total: 122ms	remaining: 7.4s
16:	learn: 2.3467236	total: 131ms	remaining: 7.48s
17:	learn: 2.3445295	total: 139ms	remaining: 7.47s
18:	learn: 2.3411195	total: 147ms	remaining: 7.46s
19:	learn: 2.3383229	total: 154ms

In [ ]:
model_lgbm = lgbm.LGBMClassifier(**study_lgbm.best_params)
model_lgbm.fit(train, target)
pred_lgbm = model_lgbm.predict_proba(test)

In [ ]:
model_svc = svm.SVC(**study_svc.best_params, probability = True)
model_svc.fit(train, target)
pred_svc = model_svc.predict_proba(test)

In [ ]:
'''
ensemble_pred = pred_svc + pred_lgbm + pred_cb + pred_xgb
top_3_ensemble = np.argsort(-ensemble_pred, axis=1)[:,:3]
print(f'Ensemble : {mapk(y_test.values.reshape(-1,1), top_3_ensemble, k=3)}')
print(f'SVC : {mapk(y_test.values.reshape(-1,1), np.argsort(-pred_xgb, axis=1)[:,:3], k=3)}')
print(f'LGBM : {mapk(y_test.values.reshape(-1,1), np.argsort(-pred_lgbm, axis=1)[:,:3], k=3)}')
print(f'CatBoost : {mapk(y_test.values.reshape(-1,1), np.argsort(-pred_cb, axis=1)[:,:3], k=3)}')
print(f'XGB : {mapk(y_test.values.reshape(-1,1), np.argsort(-pred_xgb, axis=1)[:,:3], k=3)}')
'''

In [ ]:
ensemble_pred = pred_svc + pred_lgbm + pred_cb + pred_xgb
top_3_ensemble = np.argsort(-ensemble_pred, axis=1)[:,:3]
top_3_ensemble = enc.inverse_transform(top_3_ensemble.reshape(-1, 1))
#print(top_3_pred)
top_3_ensemble = top_3_ensemble.reshape(303,3)
top_3_ensemble[:10]
pred_sub_2 = np.array([])
for i in range(303):
  pred_sub_2 = np.append(pred_sub_2, top_3_ensemble[i][0] + ' ' + top_3_ensemble[i][1] + ' ' + top_3_ensemble[i][2])
sub['prognosis'] = pred_sub_2
sub

,id,prognosis
0,707,Tungiasis Dengue Chikungunya
1,708,Dengue Chikungunya Malaria
2,709,West_Nile_fever Japanese_encephalitis Zika
3,710,Japanese_encephalitis Rift_Valley_fever Dengue
4,711,West_Nile_fever Malaria Zika
...,...,...
298,1005,Yellow_Fever Zika West_Nile_fever
299,1006,Malaria Lyme_disease Plague
300,1007,Plague Lyme_disease West_Nile_fever
301,1008,Rift_Valley_fever West_Nile_fever Japanese_enc...


In [ ]:
sub.to_csv('submission 20.csv', columns=['id', 'prognosis'], index=False)

## Maybe not now


---



In [ ]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=2000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 1e-12, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=weights)

        # Calculate the Logloss score for the weighted prediction
        score = log_loss(y_true, weighted_pred)
        return score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        self.study = optuna.create_study(sampler=sampler, study_name="OptunaWeights", direction='minimize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)

    def weights(self):
        return self.weights

In [ ]:
class Splitter:
    def __init__(self, kfold=True, n_splits=5):
        self.n_splits = n_splits
        self.kfold = kfold

    def split_data(self, X, y, random_state_list):
        if self.kfold:
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val
        else:
            raise ValueError(f"Invalid kfold: Must be True")

class Classifier:
    def __init__(self, n_estimators=100, device="cpu", random_state=0):
        self.n_estimators = n_estimators
        self.device = device
        self.random_state = random_state
        self.models = self._define_model()
        self.len_models = len(self.models)

    def _define_model(self):

        xgb_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.05,
            'max_depth': 6,
            'subsample': 0.8,
            'colsample_bytree': 0.1,
            'n_jobs': -1,
            'eval_metric': 'mlogloss',
            'objective': 'multi:softprob',
            'tree_method': 'hist',
            'verbosity': 0,
            'random_state': self.random_state,
        }
        if self.device == 'gpu':
            xgb_params['tree_method'] = 'gpu_hist'
            xgb_params['predictor'] = 'gpu_predictor'

        lgb_params = {
            'n_estimators': self.n_estimators,
            'max_depth': 6,
            'learning_rate': 0.05,
            'subsample': 0.20,
            'colsample_bytree': 0.5,
            'reg_alpha': 0.25,
            'reg_lambda': 5e-08,
            'objective': 'multiclass',
            'metric': 'multi_logloss',
            'boosting_type': 'gbdt',
            'device': self.device,
            'random_state': self.random_state
        }

        cb_params = {
            'iterations': self.n_estimators,
            'depth': 6,
            'learning_rate': 0.1,
            'l2_leaf_reg': 0.7,
            'random_strength': 0.2,
            'max_bin': 200,
            'od_wait': 65,
            'one_hot_max_size': 70,
            'grow_policy': 'Depthwise',
            'bootstrap_type': 'Bayesian',
            'od_type': 'Iter',
            'eval_metric': 'MultiClass',
            'loss_function': 'MultiClass',
            'task_type': self.device.upper(),
            'random_state': self.random_state
        }

        models = {
            'svc': svm.SVC(gamma="auto", probability=True, random_state=self.random_state),
            'xgb': xgb.XGBClassifier(**xgb_params),
            'lgb': lgbm.LGBMClassifier(**lgb_params),
            'cat': CatBoostClassifier(**cb_params),
            'brf': BalancedRandomForestClassifier(n_estimators=2000, n_jobs=-1, random_state=self.random_state),
            'rf': RandomForestClassifier(n_estimators=1000, random_state=self.random_state),
            'knn': KNeighborsClassifier(n_neighbors=5),
            'mlp': MLPClassifier(random_state=self.random_state, max_iter=1000)
        }

        return models

In [ ]:
lmao = """
from copy import deepcopy
kfold = True
n_splits = 1 if not kfold else 10
random_state = 42
random_state_list = [42]
n_estimators = 9999
early_stopping_rounds = 333
verbose = False
device = 'cpu'

splitter = Splitter(kfold=kfold, n_splits=n_splits)

# Initialize an array for storing test predictions
test_predss = np.zeros((test.shape[0], len(target.unique())))
ensemble_score = []
weights = []
trained_models = {'xgb':[], 'lgb':[], 'cat':[], 'rf':[]}
rank_df = pd.DataFrame(columns=[i for i in range(11)], index=test.index).fillna(0)


for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(train, target, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits

    # Get a set of Regressor models
    classifier = Classifier(n_estimators, device, random_state)
    models = classifier.models

    # Initialize lists to store oof and test predictions for each base model
    oof_preds = []
    test_preds = []

    # Loop over each base model and fit it to the training data, evaluate on validation data, and store predictions
    for name, model in models.items():
        if name in ['xgb', 'lgb', 'cat']:
            model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], early_stopping_rounds=early_stopping_rounds, verbose=verbose)
        else:
            model.fit(X_train_, y_train_)

        if name in trained_models.keys():
            trained_models[f'{name}'].append(deepcopy(model))

        test_pred = model.predict_proba(test)
        y_val_pred = model.predict_proba(X_val)

        score = log_loss(y_val, y_val_pred)
        print(f'{name} [FOLD-{n} SEED-{random_state_list[m]}] Logloss score: {score:.5f}')

        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)

    # Use Optuna to find the best ensemble weights
    optweights = OptunaWeights(random_state=random_state)
    y_val_pred = optweights.fit_predict(y_val.values, oof_preds)

    score = log_loss(y_val, y_val_pred)
    print(f'Ensemble [FOLD-{n} SEED-{random_state_list[m]}] Logloss score {score:.5f}')
    ensemble_score.append(score)
    weights.append(optweights.weights)

    # Predict to X_test by the best ensemble weights
    _test_preds = optweights.predict(test_preds)
    test_predss += _test_preds / (n_splits * len(random_state_list))

    # Rank Prediction
    for i in range(_test_preds.shape[0]):
        arr = _test_preds[i]
        sorted_indices = np.argsort(arr)
        for k, p in zip(range(1, 10), [9, 8, 7, 6, 5, 4, 3, 2, 1]):
            second_largest_index = sorted_indices[-k]
            rank_df.loc[i, second_largest_index] += p

    gc.collect()
"""

In [ ]:
lmao = """
mean_score = np.mean(ensemble_score)
std_score = np.std(ensemble_score)
print(f'Ensemble Logloss score {mean_score:.5f} ± {std_score:.5f}')

# Print the mean and standard deviation of the ensemble weights for each model
print('--- Model Weights ---')
mean_weights = np.mean(weights, axis=0)
std_weights = np.std(weights, axis=0)
for name, mean_weight, std_weight in zip(models.keys(), mean_weights, std_weights):
    print(f'{name}: {mean_weight:.5f} ± {std_weight:.5f}')
"""